In [1]:
import hashlib
import binascii

def generate_psk(ssid, passphrase):
    """Generate a WPA2 PSK from an SSID and a passphrase."""
    # The number of PBKDF2 iterations to perform
    iterations = 4096

    # The length of the generated PSK
    key_length = 32

    # Use PBKDF2 to generate the PSK
    psk = hashlib.pbkdf2_hmac('sha1', passphrase.encode('utf-8'), ssid.encode('utf-8'), iterations, key_length)

    # Return the PSK as a hexadecimal string
    return binascii.hexlify(psk).decode('utf-8')

# Test the function
ssid = 'testnetwork'
passphrase = '6736f7b3c8'
PMK=generate_psk(ssid, passphrase)
print("WPA: PMK - hexdump(len=32): d0 a9 86 c1 1a 6a 12 bc 7e 92 c1 db 05 08 c4 4c 44 64 4b 83 e0 75 07 58 80 d8 0b 2c 85 f8 bd 95")
print(f"Generated PMK is {PMK} the type is {type(PMK)}")



WPA: PMK - hexdump(len=32): d0 a9 86 c1 1a 6a 12 bc 7e 92 c1 db 05 08 c4 4c 44 64 4b 83 e0 75 07 58 80 d8 0b 2c 85 f8 bd 95
Generated PMK is a798824a303b9b86f3c670664da8fddb3c201e9f02d143406e2d825d29f88959 the type is <class 'str'>


In [2]:
import hashlib
import binascii

def generate_ptk(pmk, anonce, snonce, ap_mac, client_mac):
    # Combine the parameters
    data = min(ap_mac, client_mac) + max(ap_mac, client_mac) + min(anonce, snonce) + max(anonce, snonce)
    
    # Use the pseudo-random function to generate the PTK
    ptk = hashlib.pbkdf2_hmac('sha1', pmk, data, 4096, 64)
    
    # Break the PTK into the separate keys
    kck = ptk[0:16]  # Key Confirmation Key
    kek = ptk[16:32]  # Key Encryption Key
    tk = ptk[32:48]  # Temporal Key
    
    # Return the separate keys
    return kck, kek, tk
#line 3647
# Test the function
pmk = binascii.unhexlify('d0a986c11a6a12bc7e92c1db0508c44c44644b83e075075880d80b2c85f8bd95')  # Replace with your actual PMK
anonce = binascii.unhexlify('84 5a 6d 70 02 85 19 4b c8 a8 aa 77 f7 39 87 93 8a 3f 53 f6 31 ee 90 3f fc 49 48 b3 01 47 9b cd'.replace(' ',''))  # Replace with your actual ANonce
snonce = binascii.unhexlify(' b2 64 e2 09 ab 3a 06 1b e0 09 0b c1 9d 30 55 a8 31 77 02 fe 95 1b ed 8c 26 c9 bd ad 35 5c 5b 55'.replace(' ',''))  # Replace with your actual SNonce
ap_mac = binascii.unhexlify('7c:10:c9:61:69:c4'.replace(':',''))  # Replace with your actual AP MAC
client_mac = binascii.unhexlify('80:45:dd:71:d8:ce'.replace(':',''))  # Replace with your actual client MAC

kck, kek, tk = generate_ptk(pmk, anonce, snonce, ap_mac, client_mac)
print("KCK:", binascii.hexlify(kck).decode('utf-8'))
print("KEK:", binascii.hexlify(kek).decode('utf-8'))
print("TK:", binascii.hexlify(tk).decode('utf-8'))


KCK: 1aa96f6ffc7410444e6d5a36db57827e
KEK: e6fb19c061c670980e6cf770af20173e
TK: 05465a1a473a42efa5902381bf71a0d4
